In [1]:
import Dataloader
import Model
import torch
from torch.utils.data import DataLoader
import numpy as np

In [2]:
device = torch.device('cuda:0')

In [3]:
dataset = Dataloader.flickr8000_subset()

In [4]:
vocabulary_size = len(dataset.dictionary)
show_and_tell = Model.LSTM(vocabulary_size = vocabulary_size).to(device)

Initialized embedding matrices for p(y | a, y_prev)
Initialized memory state and hidden state fc layers for LSTM
Initialized soft version of attention mechanism
Beta Initialized
Initialized LSTM


In [5]:
vgg = Model.VGG()

In [6]:
vgg.vgg19 = vgg.vgg19.to(device)

In [7]:
batch_size = 64
dataloader = DataLoader(dataset,batch_size=batch_size)

In [8]:
import torch.optim as optim
from torch import nn

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(show_and_tell.parameters(),lr=0.0001)

In [10]:
words = list(dataset.dictionary.keys())

In [11]:
num_epochs = 50
for epoch in range(num_epochs):
    LOSS = []
    print(epoch)
    for images,captions,labels in dataloader:
        a = vgg(images.squeeze().to(device))
        hn,cn = show_and_tell.init_hidden(a)
        y_prev = torch.ones((a.shape[0],1,vocabulary_size)).to(device)
        
        sentence_prediction = []
        for t in range(dataset.sentence_len):
            optimizer.zero_grad()

            yt = captions.squeeze()[:,t,:].unsqueeze(dim=1).to(device)
            y_pred,hn,cn,alpha_i = show_and_tell.forward(a,y_prev,hn.to(device),cn.to(device))
            """with torch.no_grad():
                print(words[y_pred[0].argmax(1).item()])
            """
            y_prev = yt

            loss = criterion(y_pred.squeeze(1),labels[:,:,t].squeeze(1).to(device))
            
            LOSS.append(loss.detach().item())
            
            loss.backward(retain_graph=True)

            optimizer.step()
    print(np.mean(LOSS))

0
5.356482139403944
1


KeyboardInterrupt: 

In [13]:
y_pred.size()

torch.Size([64, 1, 3321])

In [15]:
labels.size()

torch.Size([64, 1, 11])

In [ ]:
input,caption,labels = dataset[3999]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(input.squeeze().permute(1,2,0).numpy())

In [ ]:
a = vgg(input.to(device))

In [ ]:
hn,cn = show_and_tell.init_hidden(a)

In [ ]:
y_prev = torch.ones((a.shape[0],1,vocabulary_size)).to(device)

In [ ]:
caption.size()

In [ ]:
yt.size()

In [ ]:
for t in range(dataset.sentence_len):
    yt = caption[:,t,:].unsqueeze(dim=1).to(device)
    y_pred,hn,cn = show_and_tell.forward(a,y_prev,hn.to(device),cn.to(device))
    with torch.no_grad():
        print(words[y_pred[0].argmax(1).item()])
    y_prev = yt

In [ ]:
a = vgg(input.squeeze().to(device))
hn,cn = show_and_tell.init_hidden(a)
y_prev = torch.ones((a.shape[0],1,vocabulary_size)).to(device)
for t in range(dataset.sentence_len):
    optimizer.zero_grad()

    yt = captions.squeeze()[:,t,:].unsqueeze(dim=1).to(device)
    y_pred,hn,cn = show_and_tell.forward(a,y_prev,hn.to(device),cn.to(device))
    with torch.no_grad():
        print(words[y_pred[0].argmax(1).item()])
    y_prev = yt